In [1]:
import pandas as pd
import numpy as np

import pytz
import astral as ast
from astral import LocationInfo
from astral.sun import sun
from datetime import datetime, timedelta, date

In [2]:
# read in with pandas
winter1 = pd.read_csv ('fok20-21data.txt')
winter2 = pd.read_csv ('fok21-22data.txt')
winter3 = pd.read_csv ('fok22-23data.txt')

In [3]:
# rename columns
winter1 = winter1.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter2 = winter2.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter3 = winter3.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})

# datetime format
winter1['datetime'] = pd.to_datetime(winter1['datetime'])
winter2['datetime'] = pd.to_datetime(winter2['datetime'])
winter3['datetime'] = pd.to_datetime(winter3['datetime'])

In [4]:
# mask out days without precipitation
codes = ['GS','IC','PL','SG','SN']

mask1 = winter1['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask2 = winter2['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask3 = winter3['wxcodes'].apply(lambda x: any(sub in x for sub in codes))

precip_days1 = winter1[mask1]
precip_days2 = winter2[mask2]
precip_days3 = winter3[mask3]

In [5]:
precip_days2

,station,datetime,precip(mm),wxcodes
215,FOK,2021-12-09 02:53:00,0.00,-SN BR
216,FOK,2021-12-09 03:53:00,0.00,-SN BR
639,FOK,2021-12-24 08:53:00,T,-SN
640,FOK,2021-12-24 09:00:00,T,-SN
641,FOK,2021-12-24 09:30:00,T,-SN
...,...,...,...,...
2537,FOK,2022-02-19 21:42:00,T,-SN
2538,FOK,2022-02-19 21:46:00,T,-SN
2539,FOK,2022-02-19 21:53:00,T,SN
2540,FOK,2022-02-19 22:04:00,T,-SN


In [6]:
# replace Trace with 0.0001
precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
precip_days3['precip(mm)'] = precip_days3['precip(mm)'].replace('T', 0.0001)

C:\Users\alyha\AppData\Local\Temp\ipykernel_37416\3060002225.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_37416\3060002225.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_37416\3060002225.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of

In [7]:
# make sure numeric form
precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
precip_days3['precip(mm)'] = pd.to_numeric(precip_days3['precip(mm)'])

C:\Users\alyha\AppData\Local\Temp\ipykernel_37416\2630444437.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_37416\2630444437.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_37416\2630444437.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

In [8]:
# eliminate special weather 
precip_days1 = precip_days1[precip_days1['datetime'].dt.minute == 53]
precip_days2 = precip_days2[precip_days2['datetime'].dt.minute == 53]
precip_days3 = precip_days3[precip_days3['datetime'].dt.minute == 53]

In [9]:
precip_days1.dtypes

station               object
datetime      datetime64[ns]
precip(mm)           float64
wxcodes               object
dtype: object

In [10]:
precip_days1['date'] = precip_days1['datetime'].dt.date

In [11]:
fok = LocationInfo(
    'Westhampton Beach', 'New York', 
    latitude= 40.8094,
    longitude= -72.6458, 
    timezone='America/New_York'
)

fok

LocationInfo(name='Westhampton Beach', region='New York', timezone='America/New_York', latitude=40.8094, longitude=-72.6458)

In [12]:
# create a date range (winter 1)
date_range1 = pd.date_range(start='2020-12-01', end='2021-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times1 = []
sunset_times1 = []

# Calculate sunrise and sunset for each date
for date in date_range1:
    s = sun(fok.observer, date=date)
    
    sunrise_times1.append(s['sunrise'])
    sunset_times1.append(s['sunset'])

# df with results
sun_times1 = pd.DataFrame({
    'datetime': date_range1.date,
    'sunrise': sunrise_times1,
    'sunset': sunset_times1
})

sun_times1['sunrise'] = pd.to_datetime(sun_times1['sunrise'])
sun_times1['sunset'] = pd.to_datetime(sun_times1['sunset'])

In [13]:
# create a date range (winter 2)
date_range2 = pd.date_range(start='2021-12-01', end='2022-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times2 = []
sunset_times2 = []

# Calculate sunrise and sunset for each date
for date in date_range2:
    s = sun(fok.observer, date=date)
    
    sunrise_times2.append(s['sunrise'])
    sunset_times2.append(s['sunset'])

# df with results
sun_times2 = pd.DataFrame({
    'datetime': date_range2.date,
    'sunrise': sunrise_times2,
    'sunset': sunset_times2
})

sun_times2['sunrise'] = pd.to_datetime(sun_times2['sunrise'])
sun_times2['sunset'] = pd.to_datetime(sun_times2['sunset'])

In [14]:
# create a date range (winter 3)
date_range3 = pd.date_range(start='2022-12-01', end='2023-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times3 = []
sunset_times3 = []

# Calculate sunrise and sunset for each date
for date in date_range3:
    s = sun(fok.observer, date=date)
    
    sunrise_times3.append(s['sunrise'])
    sunset_times3.append(s['sunset'])

# df with results
sun_times3 = pd.DataFrame({
    'datetime': date_range3.date,
    'sunrise': sunrise_times3,
    'sunset': sunset_times3
})

sun_times3['sunrise'] = pd.to_datetime(sun_times3['sunrise'])
sun_times3['sunset'] = pd.to_datetime(sun_times3['sunset'])

In [15]:
# add sunrise and sunset times (winter1)
sun_times1['sunrise'] = sun_times1['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times1['sunset'] = sun_times1['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times1['date'] = sun_times1['datetime']
sun_times1 = sun_times1.drop(columns=['datetime'])

print(sun_times1)

                sunrise               sunset        date
0   2020-12-01 06:56:18  2020-12-01 16:23:10  2020-12-01
1   2020-12-02 06:57:18  2020-12-02 16:22:57  2020-12-02
2   2020-12-03 06:58:18  2020-12-03 16:22:47  2020-12-03
3   2020-12-04 06:59:16  2020-12-04 16:22:38  2020-12-04
4   2020-12-05 07:00:13  2020-12-05 16:22:32  2020-12-05
..                  ...                  ...         ...
85  2021-02-24 06:31:56  2021-02-24 17:36:05  2021-02-24
86  2021-02-25 06:30:27  2021-02-25 17:37:15  2021-02-25
87  2021-02-26 06:28:57  2021-02-26 17:38:24  2021-02-26
88  2021-02-27 06:27:27  2021-02-27 17:39:34  2021-02-27
89  2021-02-28 06:25:56  2021-02-28 17:40:42  2021-02-28

[90 rows x 3 columns]


In [16]:
# add sunrise and sunset times (winter2)

sun_times2['sunrise'] = sun_times2['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times2['sunset'] = sun_times2['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times2['date'] = sun_times2['datetime']
sun_times2 = sun_times2.drop(columns=['datetime'])

print(sun_times2)

                sunrise               sunset        date
0   2021-12-01 06:56:03  2021-12-01 16:23:14  2021-12-01
1   2021-12-02 06:57:04  2021-12-02 16:23:00  2021-12-02
2   2021-12-03 06:58:03  2021-12-03 16:22:49  2021-12-03
3   2021-12-04 06:59:02  2021-12-04 16:22:40  2021-12-04
4   2021-12-05 06:59:59  2021-12-05 16:22:33  2021-12-05
..                  ...                  ...         ...
85  2022-02-24 06:32:17  2022-02-24 17:35:48  2022-02-24
86  2022-02-25 06:30:48  2022-02-25 17:36:58  2022-02-25
87  2022-02-26 06:29:19  2022-02-26 17:38:08  2022-02-26
88  2022-02-27 06:27:49  2022-02-27 17:39:17  2022-02-27
89  2022-02-28 06:26:18  2022-02-28 17:40:26  2022-02-28

[90 rows x 3 columns]


In [17]:
# add sunrise and sunset times (winter3)

sun_times3['sunrise'] = sun_times3['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times3['sunset'] = sun_times3['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times3['date'] = sun_times3['datetime']
sun_times3 = sun_times3.drop(columns=['datetime'])

print(sun_times3)

                sunrise               sunset        date
0   2022-12-01 06:55:48  2022-12-01 16:23:17  2022-12-01
1   2022-12-02 06:56:49  2022-12-02 16:23:03  2022-12-02
2   2022-12-03 06:57:49  2022-12-03 16:22:51  2022-12-03
3   2022-12-04 06:58:48  2022-12-04 16:22:41  2022-12-04
4   2022-12-05 06:59:45  2022-12-05 16:22:34  2022-12-05
..                  ...                  ...         ...
85  2023-02-24 06:32:38  2023-02-24 17:35:31  2023-02-24
86  2023-02-25 06:31:10  2023-02-25 17:36:41  2023-02-25
87  2023-02-26 06:29:41  2023-02-26 17:37:51  2023-02-26
88  2023-02-27 06:28:11  2023-02-27 17:39:00  2023-02-27
89  2023-02-28 06:26:40  2023-02-28 17:40:09  2023-02-28

[90 rows x 3 columns]


In [18]:
# add 'date' column to all dataframes
precip_days1['date'] = precip_days1['datetime'].dt.date
precip_days2['date'] = precip_days2['datetime'].dt.date
precip_days3['date'] = precip_days3['datetime'].dt.date

C:\Users\alyha\AppData\Local\Temp\ipykernel_37416\762931898.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['date'] = precip_days2['datetime'].dt.date


In [19]:
# merge precip days and sunrise/sunset times on date 
merged1 = pd.merge(precip_days1, sun_times1, on='date')

merged1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,FOK,2020-12-16 20:53:00,0.0001,-SN,2020-12-16,2020-12-16 07:09:04,2020-12-16 16:23:55
1,FOK,2020-12-16 21:53:00,0.5100,-SN BR,2020-12-16,2020-12-16 07:09:04,2020-12-16 16:23:55
2,FOK,2020-12-16 22:53:00,0.5100,-SN BR,2020-12-16,2020-12-16 07:09:04,2020-12-16 16:23:55
3,FOK,2020-12-16 23:53:00,0.5100,-SN BR,2020-12-16,2020-12-16 07:09:04,2020-12-16 16:23:55
4,FOK,2020-12-17 00:53:00,1.0200,SN FZFG,2020-12-17,2020-12-17 07:09:42,2020-12-17 16:24:16
...,...,...,...,...,...,...,...
74,FOK,2021-02-19 22:53:00,0.0001,-SN BR,2021-02-19,2021-02-19 06:39:06,2021-02-19 17:30:12
75,FOK,2021-02-19 23:53:00,0.0001,-SN BR,2021-02-19,2021-02-19 06:39:06,2021-02-19 17:30:12
76,FOK,2021-02-20 00:53:00,0.5100,-SN BR,2021-02-20,2021-02-20 06:37:42,2021-02-20 17:31:23
77,FOK,2021-02-20 01:53:00,0.2500,-SN BR,2021-02-20,2021-02-20 06:37:42,2021-02-20 17:31:23


In [20]:
# merge precip days and sunrise/sunset times on date 
merged2 = pd.merge(precip_days2, sun_times2, on='date')

merged2

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,FOK,2021-12-09 02:53:00,0.0000,-SN BR,2021-12-09,2021-12-09 07:03:35,2021-12-09 16:22:29
1,FOK,2021-12-09 03:53:00,0.0000,-SN BR,2021-12-09,2021-12-09 07:03:35,2021-12-09 16:22:29
2,FOK,2021-12-24 08:53:00,0.0001,-SN,2021-12-24,2021-12-24 07:13:10,2021-12-24 16:27:35
3,FOK,2021-12-24 09:53:00,0.0001,-SN BR,2021-12-24,2021-12-24 07:13:10,2021-12-24 16:27:35
4,FOK,2021-12-24 10:53:00,0.0001,-SN BR,2021-12-24,2021-12-24 07:13:10,2021-12-24 16:27:35
5,FOK,2022-01-03 18:53:00,0.0001,-SN,2022-01-03,2022-01-03 07:15:20,2022-01-03 16:35:09
6,FOK,2022-01-07 07:53:00,0.2500,-SN,2022-01-07,2022-01-07 07:15:09,2022-01-07 16:38:55
7,FOK,2022-01-07 08:53:00,1.2700,-SN BR,2022-01-07,2022-01-07 07:15:09,2022-01-07 16:38:55
8,FOK,2022-01-07 09:53:00,0.7600,-SN BR,2022-01-07,2022-01-07 07:15:09,2022-01-07 16:38:55
9,FOK,2022-01-07 10:53:00,1.2700,-SN BR,2022-01-07,2022-01-07 07:15:09,2022-01-07 16:38:55


In [21]:
# merge precip days and sunrise/sunset times on date 
merged3 = pd.merge(precip_days3, sun_times3, on='date')

merged3

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,FOK,2022-12-12 06:53:00,0.0001,-SN,2022-12-12,2022-12-12 07:05:51,2022-12-12 16:22:47
1,FOK,2022-12-12 07:53:00,0.0001,-SN BR,2022-12-12,2022-12-12 07:05:51,2022-12-12 16:22:47
2,FOK,2022-12-12 08:53:00,0.2500,-SN BR,2022-12-12,2022-12-12 07:05:51,2022-12-12 16:22:47
3,FOK,2022-12-12 09:53:00,0.2500,-SN BR,2022-12-12,2022-12-12 07:05:51,2022-12-12 16:22:47
4,FOK,2022-12-12 10:53:00,0.0001,-SN BR,2022-12-12,2022-12-12 07:05:51,2022-12-12 16:22:47
5,FOK,2022-12-12 11:53:00,0.0001,-SN BR,2022-12-12,2022-12-12 07:05:51,2022-12-12 16:22:47
6,FOK,2023-01-23 21:53:00,0.5100,-SN BR,2023-01-23,2023-01-23 07:08:41,2023-01-23 16:56:30
7,FOK,2023-01-23 22:53:00,0.5100,-SN BR,2023-01-23,2023-01-23 07:08:41,2023-01-23 16:56:30
8,FOK,2023-01-23 23:53:00,0.7600,-SN BR,2023-01-23,2023-01-23 07:08:41,2023-01-23 16:56:30
9,FOK,2023-01-31 13:53:00,0.5100,-SN BR,2023-01-31,2023-01-31 07:02:09,2023-01-31 17:06:19


In [22]:
# filter out only times after sunrise & before sunset 

days1 = merged1[(merged1['datetime'] >= merged1['sunrise']) & (merged1['datetime'] <= merged1['sunset'])]
days2 = merged2[(merged2['datetime'] >= merged2['sunrise']) & (merged2['datetime'] <= merged2['sunset'])]
days3 = merged3[(merged3['datetime'] >= merged3['sunrise']) & (merged3['datetime'] <= merged3['sunset'])]


days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
9,FOK,2020-12-17 12:53:00,5.0800,-SN FZFG,2020-12-17,2020-12-17 07:09:42,2020-12-17 16:24:16
10,FOK,2020-12-17 13:53:00,0.7600,-SN FZFG,2020-12-17,2020-12-17 07:09:42,2020-12-17 16:24:16
11,FOK,2020-12-17 14:53:00,0.2500,-SN BR,2020-12-17,2020-12-17 07:09:42,2020-12-17 16:24:16
15,FOK,2021-01-20 13:53:00,0.0001,-SN,2021-01-20,2021-01-20 07:10:19,2021-01-20 16:53:30
16,FOK,2021-01-20 14:53:00,0.0001,-SN BR,2021-01-20,2021-01-20 07:10:19,2021-01-20 16:53:30
20,FOK,2021-01-29 10:53:00,0.0001,-SN,2021-01-29,2021-01-29 07:03:33,2021-01-29 17:04:27
25,FOK,2021-02-01 07:53:00,0.5100,-SN BR,2021-02-01,2021-02-01 07:00:44,2021-02-01 17:08:10
26,FOK,2021-02-01 08:53:00,0.5100,-SN BR,2021-02-01,2021-02-01 07:00:44,2021-02-01 17:08:10
27,FOK,2021-02-01 09:53:00,0.5100,-SN BR,2021-02-01,2021-02-01 07:00:44,2021-02-01 17:08:10
28,FOK,2021-02-01 10:53:00,0.0001,-SN BR,2021-02-01,2021-02-01 07:00:44,2021-02-01 17:08:10


In [23]:
# filter out transition hours
days1['sunrise'] = pd.to_datetime(days1['sunrise'])
days1['sunset'] = pd.to_datetime(days1['sunset'])

days1['sunrise+1'] = days1['sunrise'] + timedelta(hours = 1)
days1['sunset-1'] = days1['sunset'] - timedelta(hours = 1)

days1 = days1[days1['datetime'] <= days1['sunset-1']]
days1 = days1[days1['datetime'] >= days1['sunrise+1']]



days2['sunrise'] = pd.to_datetime(days2['sunrise'])
days2['sunset'] = pd.to_datetime(days2['sunset'])

days2['sunrise+1'] = days2['sunrise'] + timedelta(hours = 1)
days2['sunset-1'] = days2['sunset'] - timedelta(hours = 1)

days2 = days2[days2['datetime'] <= days2['sunset-1']]
days2 = days2[days2['datetime'] >= days2['sunrise+1']]



days3['sunrise'] = pd.to_datetime(days3['sunrise'])
days3['sunset'] = pd.to_datetime(days3['sunset'])

days3['sunrise+1'] = days3['sunrise'] + timedelta(hours = 1)
days3['sunset-1'] = days3['sunset'] - timedelta(hours = 1)

days3 = days3[days3['datetime'] <= days3['sunset-1']]
days3 = days3[days3['datetime'] >= days3['sunrise+1']]

C:\Users\alyha\AppData\Local\Temp\ipykernel_37416\2882998222.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunrise'] = pd.to_datetime(days1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_37416\2882998222.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunset'] = pd.to_datetime(days1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_37416\2882998222.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

In [24]:
days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset,sunrise+1,sunset-1
9,FOK,2020-12-17 12:53:00,5.0800,-SN FZFG,2020-12-17,2020-12-17 07:09:42,2020-12-17 16:24:16,2020-12-17 08:09:42,2020-12-17 15:24:16
10,FOK,2020-12-17 13:53:00,0.7600,-SN FZFG,2020-12-17,2020-12-17 07:09:42,2020-12-17 16:24:16,2020-12-17 08:09:42,2020-12-17 15:24:16
11,FOK,2020-12-17 14:53:00,0.2500,-SN BR,2020-12-17,2020-12-17 07:09:42,2020-12-17 16:24:16,2020-12-17 08:09:42,2020-12-17 15:24:16
15,FOK,2021-01-20 13:53:00,0.0001,-SN,2021-01-20,2021-01-20 07:10:19,2021-01-20 16:53:30,2021-01-20 08:10:19,2021-01-20 15:53:30
16,FOK,2021-01-20 14:53:00,0.0001,-SN BR,2021-01-20,2021-01-20 07:10:19,2021-01-20 16:53:30,2021-01-20 08:10:19,2021-01-20 15:53:30
20,FOK,2021-01-29 10:53:00,0.0001,-SN,2021-01-29,2021-01-29 07:03:33,2021-01-29 17:04:27,2021-01-29 08:03:33,2021-01-29 16:04:27
26,FOK,2021-02-01 08:53:00,0.5100,-SN BR,2021-02-01,2021-02-01 07:00:44,2021-02-01 17:08:10,2021-02-01 08:00:44,2021-02-01 16:08:10
27,FOK,2021-02-01 09:53:00,0.5100,-SN BR,2021-02-01,2021-02-01 07:00:44,2021-02-01 17:08:10,2021-02-01 08:00:44,2021-02-01 16:08:10
28,FOK,2021-02-01 10:53:00,0.0001,-SN BR,2021-02-01,2021-02-01 07:00:44,2021-02-01 17:08:10,2021-02-01 08:00:44,2021-02-01 16:08:10
29,FOK,2021-02-01 11:53:00,0.2500,-SN BR,2021-02-01,2021-02-01 07:00:44,2021-02-01 17:08:10,2021-02-01 08:00:44,2021-02-01 16:08:10


In [25]:
# find total precip over each year
day_total1 = days1['precip(mm)'].sum()
day_total2 = days2['precip(mm)'].sum()
day_total3 = days3['precip(mm)'].sum()

# total number of obs each year
day_rows1 = days1.shape[0]
day_rows2 = days2.shape[0]
day_rows3 = days3.shape[0]

In [26]:
day_precip_per_hour1 = day_total1/ day_rows1
day_precip_per_hour2 = day_total2/ day_rows2
day_precip_per_hour3 = day_total3/ day_rows3


In [27]:
print('2020-2021:', day_precip_per_hour1)
print('2021-2022:', day_precip_per_hour2)
print('2022-2023:', day_precip_per_hour3)

2020-2021: 0.49819629629629625
2021-2022: 0.5185952380952381
2022-2023: 0.14005555555555554


In [28]:
# filter out only times after sunset & before NEXT sunrise 

nights1 = merged1[(merged1['datetime'] < merged1['sunrise']) | (merged1['datetime'] > merged1['sunset'])]
nights2 = merged2[(merged2['datetime'] < merged2['sunrise']) | (merged2['datetime'] > merged2['sunset'])]
nights3 = merged3[(merged3['datetime'] < merged3['sunrise']) | (merged3['datetime'] > merged3['sunset'])]

nights1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,FOK,2020-12-16 20:53:00,0.0001,-SN,2020-12-16,2020-12-16 07:09:04,2020-12-16 16:23:55
1,FOK,2020-12-16 21:53:00,0.5100,-SN BR,2020-12-16,2020-12-16 07:09:04,2020-12-16 16:23:55
2,FOK,2020-12-16 22:53:00,0.5100,-SN BR,2020-12-16,2020-12-16 07:09:04,2020-12-16 16:23:55
3,FOK,2020-12-16 23:53:00,0.5100,-SN BR,2020-12-16,2020-12-16 07:09:04,2020-12-16 16:23:55
4,FOK,2020-12-17 00:53:00,1.0200,SN FZFG,2020-12-17,2020-12-17 07:09:42,2020-12-17 16:24:16
5,FOK,2020-12-17 01:53:00,1.0200,-SN BR,2020-12-17,2020-12-17 07:09:42,2020-12-17 16:24:16
6,FOK,2020-12-17 02:53:00,1.0200,-SN BR,2020-12-17,2020-12-17 07:09:42,2020-12-17 16:24:16
7,FOK,2020-12-17 03:53:00,1.7800,-SN BR,2020-12-17,2020-12-17 07:09:42,2020-12-17 16:24:16
8,FOK,2020-12-17 04:53:00,2.2900,-SN BR,2020-12-17,2020-12-17 07:09:42,2020-12-17 16:24:16
12,FOK,2020-12-17 18:53:00,0.0001,-SN,2020-12-17,2020-12-17 07:09:42,2020-12-17 16:24:16


In [29]:
# filter out transition hours
nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
nights1['sunset'] = pd.to_datetime(nights1['sunset'])

nights1['sunrise-1'] = nights1['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights1['sunset+1'] = nights1['sunset'] + timedelta(hours = 1)

nights1 = nights1[nights1['datetime'] >= nights1['sunset+1']]
nights1 = nights1[nights1['datetime'] <= nights1['sunrise-1']]



nights2['sunrise'] = pd.to_datetime(nights2['sunrise'])
nights2['sunset'] = pd.to_datetime(nights2['sunset'])

nights2['sunrise-1'] = nights2['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights2['sunset+1'] = nights2['sunset'] + timedelta(hours = 1)

nights2 = nights2[nights2['datetime'] >= nights2['sunset+1']]
nights2 = nights2[nights2['datetime'] <= nights2['sunrise-1']]



nights3['sunrise'] = pd.to_datetime(nights3['sunrise'])
nights3['sunset'] = pd.to_datetime(nights3['sunset'])

nights3['sunrise-1'] = nights3['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights3['sunset+1'] = nights3['sunset'] + timedelta(hours = 1)

nights3 = nights3[nights3['datetime'] >= nights3['sunset+1']]
nights3 = nights3[nights3['datetime'] <= nights3['sunrise-1']]


C:\Users\alyha\AppData\Local\Temp\ipykernel_37416\36922422.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_37416\36922422.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunset'] = pd.to_datetime(nights1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_37416\36922422.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

In [30]:
# find total precip over each year
night_total1 = nights1['precip(mm)'].sum()
night_total2 = nights2['precip(mm)'].sum()
night_total3 = nights3['precip(mm)'].sum()

# total number of obs each year
night_rows1 = nights1.shape[0]
night_rows2 = nights2.shape[0]
night_rows3 = nights3.shape[0]

In [31]:
night_precip_per_hour1 = night_total1/ night_rows1
night_precip_per_hour2 = night_total2/ night_rows2
night_precip_per_hour3 = night_total3/ night_rows3

In [32]:
print('2020-2021:', night_precip_per_hour1)
print('2021-2022:', night_precip_per_hour2)
print('2022-2023:', night_precip_per_hour3)

2020-2021: 0.6704360000000001
2021-2022: 0.10006
2022-2023: 0.5933333333333334


In [33]:
data = pd.concat([days1, nights1, days2, nights2, days3, nights3])

data['date'] = pd.to_datetime(data['date'])
data['month'] = data['date'].dt.month

data['state'] = 'NY'

data.to_csv('SO.fok.csv', index=False)